# Ensemble

Todo:

- Remove max evals here - it doesn't work

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import mlflow
import os

from datetime import datetime, timedelta

from nb_utils import train_eval_plot_ensemble

In [ ]:
with open('../config/sample_homogeneous_train_config.json') as f_train, \
    open('../config/sample_homogeneous_test_config.json') as f_test, \
    open('../config/sample_homogeneous_forecast_config.json') as f_forecast:
    default_train_config = json.load(f_train)
    default_test_config = json.load(f_test)
    default_forecast_config = json.load(f_forecast)

In [ ]:
with open('mlflow_credentials.json') as f_cred:
    mlflow_credentials = json.load(f_cred)

In [ ]:
os.environ['MLFLOW_TRACKING_USERNAME'] = mlflow_credentials['username']
os.environ['MLFLOW_TRACKING_PASSWORD'] = mlflow_credentials['password']

mlflow.set_tracking_uri("http://ec2-54-175-207-176.compute-1.amazonaws.com")
mlflow.set_experiment("SEIHRD_ENSEMBLE_V0")

In [ ]:
model_class = 'homogeneous_ensemble'
default_train_config['model_class'] = model_class
default_test_config['model_class'] = model_class
default_forecast_config['model_class'] = model_class

default_train_config['model_parameters']['n'] = 10
default_train_config['search_parameters']['child_model']['max_evals'] = 10
default_train_config['search_parameters']['ensemble_model']['max_evals'] = 10
default_forecast_config['model_parameters']['uncertainty_parameters']['date_of_interest'] = '6/25/20'
default_forecast_config['model_parameters']['uncertainty_parameters']['percentiles'] = [10,50,90]
default_forecast_config['model_parameters']['uncertainty_parameters']['ci'] = []

In [ ]:
current_day = datetime.now().date() - timedelta(4) # Forecast starts from the day after current day
forecast_length = 30 # Length of forecast interval (days)

### Inputs to train-eval-plot

In [ ]:
region = ['delhi']
region_type = 'state'

current_day = datetime.now().date() - timedelta(40)
forecast_length = 30

name_prefix = region

### Train eval plot

In [ ]:
# remove max_evals here, it doesn't work
params, metrics, train1_params, train2_params = train_eval_plot_ensemble(region, region_type, 
                                                current_day, forecast_length,
                                                default_train_config, default_test_config, default_forecast_config,
                                                train_period = 14, test_period = 7,
                                                max_evals = 100, data_source = 'rootnet_stats_history', 
                                                mlflow_log = False, mlflow_run_name = "Ensemble testing")